# Recommender systems

- Implementing collaborative filtering and latent factors


In [3]:
import numpy as np
import pandas as pd
import os.path
import random
from random import randint
from random import uniform
import operator
from operator import itemgetter
print(np.version.version)

1.16.5


In [44]:
# -*- coding: utf-8 -*-
"""
### NOTES
This file is an example of what your code should look like. It is written in Python 3.6.
To know more about the expectations, please refer to the guidelines.
"""

#####
##
## DATA IMPORT
##
#####

#Where data is located
movies_file = '../data/movies.csv'
users_file = '../data/users.csv'
ratings_file = '../data/ratings.csv'
predictions_file = '../data/predictions.csv'
submission_file = '../data/submission.csv'

# movies_file = r'/prediction/data/movies.csv'
# users_file = '/prediction/data/users.csv'
# ratings_file = '/prediction/data/ratings.csv'
# predictions_file = '/prediction/data/predictions.csv'
# submission_file = '/data/submission.csv'

# Read the data using pandas
movies_description = pd.read_csv(movies_file, delimiter=';', 
                                 dtype={'movieID':'int', 'year':'int', 'movie':'str'}, names=['movieID', 'year', 'movie'])
users_description = pd.read_csv(users_file, delimiter=';', 
                                dtype={'userID':'int', 'gender':'str', 'age':'int', 'profession':'int'}, names=['userID', 'gender', 'age', 'profession'])
ratings_description = pd.read_csv(ratings_file, delimiter=';', 
                                  dtype={'userID':'int', 'movieID':'int', 'rating':'int'}, names=['userID', 'movieID', 'rating'])
predictions_description = pd.read_csv(predictions_file, delimiter=';', names=['userID', 'movieID'], header=None)



#####
##
## COLLABORATIVE FILTERING
##
#####

def cosine_similarity(a, b):
    denominator = np.linalg.norm(a)*np.linalg.norm(b)
    if denominator == 0:
        return 0
    
    return np.dot(a, b) / denominator


def similarity_matrix_with_cosine(ratingMatrix):
    number_items = np.shape(ratingMatrix)[0]
    similarity_matrix = np.zeros((number_items, number_items), dtype=object)
    
    for i in range(number_items):
        print(i)
        for j in range(i, number_items):
            
            movieID = j + 1
            similarity_matrix[i][j] = (movieID, cosine_similarity(ratingMatrix[:, i], ratingMatrix[:, j]))
            similarity_matrix[j][i] = similarity_matrix[i][j]
   

        
    return similarity_matrix

def predict_collaborative_filtering(movies, users, ratings, predictions):
    
    # Processing predictions data in order to return it from this function
    number_predictions = len(predictions)
    prediction_creating = [[idx, random.uniform(0, 5)] for idx in range(1, number_predictions + 1)]
    predictions_ratings = pd.DataFrame(prediction_creating, columns = ['Id', 'Rating'])
    predictions_ratings['movieID'] = predictions['movieID']
    predictions_ratings['userID'] = predictions['userID']
    
    # Adding missing movie_ids to the numpy arrays
    range_missing = range(3696, 3707)
    
    '''
    Creating utility matrix 'u' : User x Movie -> Rating
    '''        
    utility_matrix = ratings.pivot_table(index='movieID', columns='userID', values='rating', fill_value=0)
    
    original_rating = utility_matrix.values
    for i, row in utility_matrix.iterrows():
        if(i in range_missing):
            original_rating = np.vstack([original_rating, row.values])
            
    
    '''
    Creating matrix for cosine similarity
    '''
    r = ratings \
    .groupby('movieID', as_index=False, sort=False) \
    .mean() \
    .rename(columns={'movieID': 'movieID', 'rating' : 'mean_rating'})
    r.drop('userID', axis=1, inplace=True)
    
    new_r = ratings.merge(r, how='left', on='movieID', sort=False)
    new_r['centered_cosine'] = new_r['rating'] - new_r['mean_rating']
    
    centered_cosine = new_r \
    .pivot_table(index='movieID', columns='userID', values='centered_cosine') \
    .fillna(0)
    
    
    all_movies_numpy = centered_cosine.values
    for i, row in centered_cosine.iterrows():
        if(i in range_missing):
            all_movies_numpy = np.vstack([all_movies_numpy, row.values])
            
    # Similarity matrix
    similarity_matrix = similarity_matrix_with_cosine(all_movies_numpy)
            
            
    '''
    Cosine similarity - find similar users for a certain user based on |N|,
    also making a prediction with Pearson correlation
    '''
    for i, user_movie in predictions.iterrows():
        current_rating = original_rating[user_movie['movieID'] - 1][user_movie['userID'] - 1]
    #     if (current_rating > 0):
    #         predictions_ratings.at[i, 'Rating'] = current_rating
    #         continue
        
        # Get N similar items
        #top_N_similar_movies = similarity_matrix[user_movie['movieID'] - 1]
        #top_N_similar_movies.sort(key=lambda pair: pair[1], reverse=True)
        top_N_similar_movies = sorted(similarity_matrix[user_movie['movieID'] - 1], key=lambda pair: pair[1], reverse=True)
        similar_movies = top_N_similar_movies[1:4]
        
        #Predicting the rating with Pearson correlation
        pearson_denominator = 0
        #sum([pair[1] for pair in similar_movies])
        for i, pair in enumerate(similar_movies):
            if(original_rating[similar_movies[i][0] - 1][user_movie['userID'] - 1] != 0):
                pearson_denominator += similar_movies[i][1]
        
        pearson_numerator = 0
        for i in range(0, 3):
            if(original_rating[similar_movies[i][0] - 1][user_movie['userID'] - 1] != 0):
                print(original_rating[similar_movies[i][0] - 1][user_movie['userID'] - 1])
                pearson_numerator += similar_movies[i][1] * original_rating[similar_movies[i][0] - 1][user_movie['userID'] - 1]
        
        if(pearson_denominator == 0):
            predictions_ratings.at[i, 'Rating'] = 3.5813
            print("---")
            print("Predicted rating : ", predictions_ratings.at[i, 'Rating'])
            print("---")
            continue
            
        final_prediction = (pearson_numerator / pearson_denominator)
        if(final_prediction < 1):
            predictions_ratings.at[i, 'Rating'] = 3.5813
        else:
            predictions_ratings.at[i, 'Rating'] = final_prediction
        print("---")
        print("Predicted rating : ", predictions_ratings.at[i, 'Rating'])
        print("---")
    
    return predictions_ratings


#####
##
## LATENT FACTORS
##
#####

def predict_latent_factors(movies, users, ratings, predictions):
    ## TO COMPLETE
    
    # Processing predictions data in order to return it from this function
    number_predictions = len(predictions)
    prediction_creating = [[idx, random.uniform(0, 5)] for idx in range(1, number_predictions + 1)]
    predictions_ratings = pd.DataFrame(prediction_creating, columns = ['Id', 'Rating'])
    predictions_ratings['movieID'] = predictions['movieID']
    predictions_ratings['userID'] = predictions['userID']
    
    # Adding missing movie_ids to the numpy arrays
    range_missing = range(3696, 3707)
    
    '''
    Creating utility matrix 'u' : User x Movie -> Rating
    '''        
    utility_matrix = ratings.pivot_table(index='movieID', columns='userID', values='rating', fill_value=0)
    
    original_rating = utility_matrix.values
    for i, row in utility_matrix.iterrows():
        if(i in range_missing):
            original_rating = np.vstack([original_rating, row.values])
            
    '''
    Utility matrix with subtracted mean of the rating per movie
    '''
    r = ratings \
    .groupby('movieID', as_index=False, sort=False) \
    .mean() \
    .rename(columns={'movieID': 'movieID', 'rating' : 'mean_rating'})
    r.drop('userID', axis=1, inplace=True)
    
    new_r = ratings.merge(r, how='left', on='movieID', sort=False)
    new_r['centered_cosine'] = new_r['rating'] - new_r['mean_rating']
    
    centered_cosine = new_r \
    .pivot_table(index='movieID', columns='userID', values='centered_cosine') \
    .fillna(0)
    
    
    all_movies_numpy = centered_cosine.values
    for i, row in centered_cosine.iterrows():
        if(i in range_missing):
            all_movies_numpy = np.vstack([all_movies_numpy, row.values])
    
    # Doing Matrix factorization Q * PT
    U, S, VT = np.linalg.svd(original_rating, full_matrices=False)
    
    Q = U
    S_diagonal = np.diag(S)
    P = S_diagonal.dot(VT)
    
    # Predicting rating
    for i, user_movie in predictions.iterrows():
            
        qi = Q[user_movie['movieID'] - 1, :]
        px = P[:, user_movie['userID'] - 1]
        
        print(qi.dot(px))
        predictions_ratings.at[i, 'Rating'] = qi.dot(px)
 
    
    return predictions_ratings
    
    
  
    


#####
##
## FINAL PREDICTORS
##
#####

def predict_final(movies, users, ratings, predictions):
    ## TO COMPLETE

    pass


# rating_predictions = predict_collaborative_filtering(movies_description,
#                                                      users_description, ratings_description, predictions_description)
# rating_predictions.drop('userID', axis=1, inplace=True)
# rating_predictions.drop('movieID', axis=1, inplace=True)

latent_factors_ratings = predict_latent_factors(movies_description,
                                                users_description, ratings_description, predictions_description)

#####
##
## RANDOM PREDICTORS
## //!!\\ TO CHANGE
##
#####

#By default, predicted rate is a random classifier
def predict_random(movies, users, ratings, predictions):
    number_predictions = len(predictions)

    return [[idx, randint(1, 5)] for idx in range(1, number_predictions + 1)]

#####
##
## SAVE RESULTS
##
#####    


# ## //!!\\ TO CHANGE by your prediction function
# submission_read = pd.read_csv(submission_file)
# submission_read.columns = ['id', 'rating']

# # predictions = predict_random(movies_description, users_description, ratings_description, predictions_description)
# # print(predictions)
# # predictions_df = pd.DataFrame(predictions, columns = ['Id', 'Rating'])

# submission_result = submission_read.merge(rating_predictions, how='left', left_on='id', right_on='Id')
# submission_result.drop('id', axis=1, inplace=True)
# submission_result.drop('rating', axis=1, inplace=True)
# submission_result.head()
# submission_result.to_csv('submission_nn.csv', index=False)

0.659090909090908
0.6716839134524931
-1.0408340855860843e-15
0.49425287356321446
2.2898349882893854e-16
-5.828670879282072e-16
-0.16666666666666619
-7.077671781985373e-16
-1.1137724550898218
9.992007221626409e-16
5.134781488891349e-16
9.71445146547012e-17
-7.598088824778415e-16
-3.885780586188048e-16
-5.048045315092509e-16
2.220446049250313e-16
1.6653345369377348e-15
-2.42861286636753e-16
3.0531133177191805e-16
-4.787836793695988e-16
-2.1788126858268697e-15
6.938893903907228e-17
-1.624060150375941
0.843
7.563394355258879e-16
5.204170427930421e-16
-2.706168622523819e-16
7.008282842946301e-16
-1.9914625504213745e-15
7.632783294297951e-16
-0.3468809073724053
-9.992007221626409e-16
7.494005416219807e-16
1.1986939218999737e-15
-0.846726190476191
-2.4077961846558082e-15
1.8596235662471372e-15
1.7763568394002505e-15
0.5704989154013008
2.949029909160572e-17
-2.55351295663786e-15
-6.453171330633722e-16
1.4988010832439613e-15
-1.3877787807814457e-16
6.661338147750939e-16
1.1934897514720433e-15
-

-6.38378239159465e-16
-8.673617379884035e-17
1.512789281364192
-1.1796119636642288e-15
-3.469446951953614e-16
-2.5118795932144167e-15
-8.326672684688674e-17
5.967448757360216e-16
2.4702462297909733e-15
5.551115123125783e-17
-6.938893903907228e-17
-1.214306433183765e-15
1.2836953722228372e-15
6.938893903907228e-16
2.345346139520643e-15
0.0
6.036837696399289e-16
9.992007221626409e-16
-1.0547118733938987e-15
-1.2455314557513475e-15
2.095545958979983e-15
2.688821387764051e-16
0.1770186335403713
-0.2522255192878353
-1.27675647831893e-15
2.7755575615628914e-16
3.122502256758253e-16
-2.636779683484747e-16
1.8735013540549517e-16
1.7208456881689926e-15
3.9898639947466563e-16
-6.38378239159465e-16
1.6653345369377348e-16
6.314393452555578e-16
1.8041124150158794e-16
-2.706168622523819e-16
-1.6653345369377348e-16
-4.440892098500626e-16
1.942890293094024e-16
-5.134781488891349e-16
2.192690473634684e-15
1.6653345369377348e-15
-2.458970527197124e-16
3.37403716077489e-16
3.7470027081099033e-16
9.992007

1.6653345369377348e-16
-1.0547118733938987e-15
4.163336342344337e-16
6.245004513516506e-16
-1.97758476261356e-16
9.159339953157541e-16
1.429412144204889e-15
2.8449465006019636e-16
5.273559366969494e-16
8.326672684688674e-16
4.917941054394248e-16
8.326672684688674e-16
4.718447854656915e-16
6.938893903907228e-17
-3.122502256758253e-16
4.163336342344337e-17
4.996003610813204e-16
-7.632783294297951e-16
2.3071822230491534e-16
-0.6009389671361526
8.049116928532385e-16
-1.3322676295501878e-15
-5.759281940243e-16
-4.0245584642661925e-16
-5.718052790864933e-17
6.106226635438361e-16
1.0547118733938987e-15
-1.061650767297806e-15
-1.1796119636642288e-15
-3.0531133177191805e-16
-1.2212453270876722e-15
-1.4155343563970746e-15
3.122502256758253e-16
4.544975507059235e-16
-1.196959198423997e-16
4.458239333260394e-16
1.2695035460992916
0.5049019607843104
-1.6653345369377348e-16
1.2923689896027213e-16
-2.7755575615628914e-17
4.85722573273506e-16
-6.800116025829084e-16
2.3592239273284576e-16
-7.9103390504

8.187894806610529e-16
4.510281037539698e-16
-4.85722573273506e-16
-3.469446951953614e-16
4.0245584642661925e-16
8.881784197001252e-16
-4.0939474033052647e-16
6.995272416876475e-16
3.937822290467352e-16
0.39024390243902424
-0.015294117647063496
-1.3877787807814457e-16
1.186550857568136e-15
9.71445146547012e-16
2.7755575615628914e-16
-2.0816681711721685e-16
1.3670490093847776
-3.0531133177191805e-16
-1.915134717478395e-15
-8.604228440844963e-16
0.10828025477706825
-9.194034422677078e-17
2.220446049250313e-16
-1.3877787807814457e-17
4.718447854656915e-16
9.992007221626409e-16
-2.7755575615628914e-16
-9.818534874028728e-16
3.219646771412954e-15
6.83481049534862e-16
-1.3877787807814457e-15
5.551115123125783e-17
-2.31755233494364
-1.3877787807814457e-17
6.522560269672795e-16
-0.18072289156626448
-8.881784197001252e-16
8.049116928532385e-16
-7.355227538141662e-16
1.3322676295501878e-15
-4.440892098500626e-16
-6.557254739192331e-16
1.4198711650870166e-15
-6.661338147750939e-16
1.41553435639707

-5.440092820663267e-15
-1.942890293094024e-16
4.718447854656915e-16
-0.7619047619047606
-3.0531133177191805e-16
0.06463878326996253
-1.5265566588595902e-16
-2.192690473634684e-15
3.6801088252015197e-16
6.800116025829084e-16
-4.579669976578771e-15
-1.6421052631578952
-0.3780821917808216
-1.366430260047282
-7.494005416219807e-16
1.8041124150158794e-15
1.3877787807814457e-17
-4.0592529337857286e-16
-8.534839501805891e-16
9.853229343548264e-16
-5.412337245047638e-16
-1.4155343563970746e-15
-8.94930915692112e-18
3.7470027081099033e-16
-0.9974413646055427
4.440892098500626e-16
0.11801242236024893
6.158268339717665e-16
0.6011235955056204
4.0939474033052647e-16
6.661338147750939e-16
1.8127860323957634e-15
-3.3306690738754696e-16
-2.0816681711721685e-16
-4.718447854656915e-16
1.3877787807814457e-15
4.440892098500626e-16
7.91033905045424e-16
1.5543122344752192e-15
-3.4763858458575214e-15
0.030581039755351452
1.1102230246251565e-15
-5.134781488891349e-16
1.249000902703301e-16
-2.0816681711721685e

...

-5.898059818321144e-17
1.159604519774014
6.175615574477433e-16
1.734723475976807e-16
3.608224830031759e-16
-0.07389162561576301
0.0457610789980706
-3.3306690738754696e-16
-1.1934897514720433e-15
-7.806255641895632e-17
9.71445146547012e-17
-6.38378239159465e-16
8.604228440844963e-16
-9.71445146547012e-17
-2.220446049250313e-16
7.008282842946301e-16
-1.4155343563970746e-15
-0.13913043478261186
1.1934897514720433e-15
-5.620504062164855e-16
2.671474153004283e-16
1.6653345369377348e-15
-1.3322676295501878e-15
1.4988010832439613e-15
0.39317073170731814
0.4956097560975564
-1.0234868508263162e-16
-7.632783294297951e-17
-9.71445146547012e-17
-6.314393452555578e-16
2.3592239273284576e-16
3.2751579226442118e-15
6.036837696399289e-16
-1.790234627208065e-15
-4.579669976578771e-16
2.3592239273284576e-16
-0.1041990668740288
4.217133407852058e-16
-1.6653345369377348e-16
-1.3877787807814457e-17
-2.4424906541753444e-15
0.9368836291913218
-3.3480163086352377e-16
4.5102810375396984e-17
-4.440892098500626e

-1.249000902703301e-16
-1.3183898417423734e-16
-1.0217521273503394e-15
-9.298117831235686e-16
-1.1102230246251565e-16
-1.6653345369377348e-16
8.534839501805891e-16
2.0816681711721685e-17
1.0547118733938987e-15
3.552713678800501e-15
8.881784197001252e-16
3.95516952522712e-16
-7.563394355258879e-16
-1.6653345369377348e-15
-1.7235421166306768
-4.787836793695988e-16
6.800116025829084e-16
1.5265566588595902e-15
5.377642775528102e-16
-2.7755575615628914e-16
-4.208179784203956e-17
0.06407035175879461
1.1796119636642288e-16
-4.198030811863873e-16
-4.649058915617843e-16
-5.967448757360216e-16
-7.771561172376096e-16
-3.0531133177191805e-16
4.371503159461554e-16
-1.4710455076283324e-15
2.220446049250313e-16
0.0
-1.1102230246251565e-15
-4.163336342344337e-17
-2.5370330836160804e-17
-5.82433407059213e-16
-2.7755575615628914e-16
0.5869853917662692
-8.257283745649602e-16
3.885780586188048e-16
2.0643209364124004e-16
0.08695652173913179
0.7026239067055412
-4.961309141293668e-16
5.898059818321144e-17
4.

-7.077671781985373e-16
-1.1102230246251565e-16
1.942890293094024e-16
-3.469446951953614e-17
-2.4980018054066022e-15
7.632783294297951e-17
-4.718447854656915e-16
-1.1796119636642288e-16
-1.3302151543498617
-5.551115123125783e-16
1.5265566588595902e-15
0.2937365010799154
-1.457167719820518e-16
0.29805615550756187
8.881784197001252e-16
-5.412337245047638e-16
-7.112366251504909e-17
-0.8816821097647898
-2.671474153004283e-16
-3.469446951953614e-17
-2.1094237467877974e-15
3.469446951953614e-16
-1.6852791878172597
3.497202527569243e-15
-3.937822290467352e-16
3.122502256758253e-16
-5.551115123125783e-16
-5.689893001203927e-16
3.066991105526995e-15
5.828670879282072e-16
3.642919299551295e-16
-7.632783294297951e-16
-1.6930901125533637e-15
3.469446951953614e-17
-5.48172618408671e-16
8.881784197001252e-16
-4.163336342344337e-17
-2.220446049250313e-16
2.0192181260370035e-15
9.367506770274758e-16
-1.0408340855860843e-16
-1.6306400674181987e-16
8.049116928532385e-16
-0.18604651162790667
0.90377588306

3.885780586188048e-16
1.6340711515662156e-16
1.1102230246251565e-16
5.724587470723463e-17
-1.5265566588595902e-15
1.0515789473684223
-7.077671781985373e-16
8.049116928532385e-16
-1.457167719820518e-16
-1.6306400674181987e-16
2.248201624865942e-15
-2.983724378680108e-16
1.9081958235744878e-16
-0.9464285714285714
2.831068712794149e-15
9.43689570931383e-16
-7.788908407135864e-16
0.5758447160316296
-6.993104012531504e-17
1.0824674490095276e-15
1.1796119636642288e-16
1.687018580387445e-16
-2.8449465006019636e-16
1.3877787807814457e-15
-4.631711680858075e-16
2.7755575615628914e-17
-5.741934705483231e-16
0.9300911854103363
3.0184188481996443e-16
1.222980050563649e-16
4.163336342344337e-17
0.6046511627906974
-4.773959005888173e-15
-4.1772141301521515e-15
-0.18826405867970836
9.280770596475918e-17
-8.049116928532385e-16
4.163336342344337e-16
1.2212453270876722e-15
-0.0728562217923922
-1.3322676295501878e-15
1.5265566588595902e-16
-2.185751579730777e-16
-3.365363543395006e-16
-4.218847493575595e

-1.1003271537622683
-9.71445146547012e-16
3.41740524767431e-16
3.0531133177191805e-16
-2.220446049250313e-16
-2.914335439641036e-16
-1.2524703496552547e-15
-6.245004513516506e-16
1.1934897514720433e-15
1.5751289161869408e-15
1.457167719820518e-15
-1.7486012637846216e-15
-7.355227538141662e-16
-0.08888888888889199
8.326672684688674e-16
-1.2524703496552547e-15
5.551115123125783e-17
-1.1102230246251565e-16
8.413408858487514e-17
2.6020852139652106e-17
-1.3877787807814457e-16
3.0531133177191805e-15
3.608224830031759e-16
6.800116025829084e-16
-8.326672684688674e-16
-2.3071822230491534e-16
-4.0245584642661925e-16
-4.163336342344337e-16
8.81239525796218e-16
8.326672684688674e-17
1.2073675392798577e-15
-1.2212453270876722e-15
3.469446951953614e-16
0.7630853994490354
-1.6653345369377348e-16
9.228728892196614e-16
-0.7999999999999997
-1.8041124150158794e-16
-1.5105194398358113e-17
-7.771561172376096e-16
4.163336342344337e-17
-9.506284648352903e-16
-4.996003610813204e-16
-1.3322676295501878e-15
4.6

In [86]:
submission_read = pd.read_csv(submission_file)
submission_read.to_csv('submission_test.csv', index=False)